## Analyzing the bachelorette data set

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data = pd.read_csv('..//input//bachelorettedataset//BacheloretteDSFinal-Dogu.csv')

In [ ]:
data.head()

In [ ]:
data["Season"].unique()   # to see how many seasons the data covers

In [ ]:
# cleaning the data

In [ ]:
data.isnull().sum() # checking out missing values

In [ ]:
# handling missing values

In [ ]:
data = data[~data['Name'].isnull()]  # dropping rows with no data, which all have missing name

In [ ]:
data.loc[data['Height (cm)'].isnull(),['Height (cm)']] = data['Height (cm)'].mean() # filling missing height info with mean

In [ ]:
data.loc[data['College'].isnull(),'College'] = data['College'].value_counts().index[0] # filling in with most common college

In [ ]:
data.isnull().sum() # better

In [ ]:
data.info()

In [ ]:
data[data['Win_Loss']==1]

Because of the fact that out of 141 contestant only 5 were chosen, a ML model would not have enough data to work with ( might as well try to guess ). So we'll only stick to analyzing the data.

In [ ]:
# replace outcome with "Yes" or "No"

data['Win_Loss'] = data['Win_Loss'].astype(object)
data.replace({'Win_Loss':{'1.0':'Yes','0.0':'No'}},inplace=True)

In [ ]:
#cleaning up a little 

data['Age'] = data["Age"].astype('int')
data['Height (cm)'].round(decimals=2)


In [ ]:
print(data['Hometown'].nunique())
print(data['Occupation'].nunique())
print(data['College'].nunique())
print(data['State'].nunique())

In [ ]:
# dropping cols that have almost no common values

data.drop(['Hometown','Occupation','College'],axis=1,inplace=True)

Doing some clustering

Because we are dealing with both numerical ( age / height ) and categorical ( state, color of hair / eyes)  , K means will not work. Instead we will use K -modes which can work with categorical data. If we want to add the numerical data into our analysis with K-modes, we have to import KPrototypes.

more info here: https://github.com/nicodv/kmodes
original paper here: https://pdfs.semanticscholar.org/d42b/b5ad2d03be6d8fefa63d25d02c0711d19728.pdf

In [ ]:
from kmodes.kprototypes import KPrototypes

# determine no of clusters with elbow method
cost = []
K = range(1,7)
for num_clusters in list(K):
    kproto = KPrototypes(n_clusters=num_clusters, init = "Cao")
    kproto.fit_predict(data.drop(['Name'],axis=1), categorical=[0,2,3,5,6,7])
    cost.append(kproto.cost_)
    
plt.plot(K, cost, 'bx-')
plt.xlabel('k clusters')
plt.ylabel('Cost')
plt.title('Elbow Method For Optimal k')
plt.show()

In [ ]:
# let's try 5
kproto = KPrototypes(n_clusters=5, init='Cao')
clusters = kproto.fit_predict(data.drop(['Name'],axis=1), categorical=[0,2,3,5,6,7]) #define the cols which are categorical

In [ ]:
data['cluster'] = clusters

In [ ]:
data.head()

In [ ]:
#print the cluster groups
gb = data[['Name','cluster']].groupby('cluster')
for key, item in gb:
    print(gb.get_group(key), "\n\n")

In [ ]:
# another way to see the clusters
# for s, c in zip(data['Name'], clusters):
#     print("Name: {}, cluster:{}".format(s, c))

In [ ]:
for c in range(0,5):
    print("Cluster {} is composed of {} people".format(c, data[data['cluster']==c].shape[0]))

In [ ]:
#rough visio of the clusters

fig, ax= plt.subplots(figsize=(30,10))
ax.scatter(data['Name'], clusters, c=clusters, s=200)
ax.set_ylabel('Cluster')
ax.set_xticks('')
ax.set_yticklabels(['','0','1','2','3','4']) # i'll keep the same cluster names as in the print above
ax.set_ylim(-1,6)
for x,y in zip(data['Name'],clusters):

    label = "{}".format(x)

    plt.annotate(label, 
                 (x,y),
                 textcoords="offset points",
                 xytext=(0,10), 
                 ha='left',
                 rotation=45)

plt.show()

Dimension reduction

Again, a PCA analysis won't work on our data becuase of the categorical variables
In this case, the way to go is with a Factor Analysis of Mixed Data (FAMD)

We will use the prince library which can do a FAMD in Python

More about it here: https://nextjournal.com/pc-methods/calculate-pc-mixed-data?change-id=CWQNw1kVRgQMFFzobMC2bo&node-id=d4243af6-f940-41fc-8ffa-a235bc135601
https://github.com/MaxHalford/prince

In [ ]:
pip install prince

In [ ]:
import prince

In [ ]:
famd = prince.FAMD(
     n_components=5,
     n_iter=10,
     copy=True,
     check_input=True,
     engine='auto',       ## Can be "auto", 'sklearn', 'fbpca'
     random_state=42)

In [ ]:
famd = famd.fit(data.drop('Win_Loss', axis=1)) ## Exclude target variable

In [ ]:
print(famd.explained_inertia_)

In [ ]:
# The first dimmension stands for 55 % of variance within the data
# Will update the notebook as soon as I can
# To Do : Categorize eveyrthing and do a MCA